# Voorbeeld voor CNN's 
Een klein voorbeeldje voor de basis van CNN's. Het idee is om de terminologie uit deel 1 hier toe te passen. Laten we beginnen bij de input.

De input van een CNN heeft vaak rank = 4. Dit betekent dat we te maken hebben met axis 0, axis 1, axis 2 en axis 3 (oftwel A0, A1, A2 en A3). Laten we de input van achter naar voor bekijken:
- (A2, A3) zijn de hoogte en breedte van een afbeelding. Deze worden vaak gekozen als $24\times24$ of $224\times224$ pixels.
- A1 staat voor de kleur kanalen. Een kleuren afbeelding bestaat uit 3 kanalen Roodwaardes, Groenwaardes en Blauwwaardes (oftewel RGB). Een zwartwit afbeelding bestaat uit 1 kanaal, namelijk Grijswaardes (of grayscale).<br> <br>
Door A1, A2 en A3 te combineren kan je een individuele pixelwaarde (lees scalar) krijgen. De laatste axis(A0) staat voor de Batchgroote. Om je data te trainen gebruik je niet 1 afbeelding, maar heel veel afbeeldingen --> Batches.

Wanneer je 1 grayscale afbeelding van 28x28 (shape: [1, 1, 28, 28]) hebt en die door een CNN layer gaat zal dit zorgen voor een verandering van pixelwaardes, maar misschien ook de lengte van A1, A2 en A3. Dit ligt aan het aantal filters dat over de afbeelding gaat. Gaan hier 3 filters overheen dan zal de output 3 kanalen hebben. De resultaten van filters over een afbeelding heten *feature maps*.

# PyTorch Tensors

In [3]:
import torch
import numpy as np

In [4]:
t = torch.Tensor()

In [5]:
print(t.dtype)
print(t.device)
print(t.layout) # voor meer info-> zie wiki pagina: stride of an array

torch.float32
cpu
torch.strided


Let op bij PyTorch tensors dat de data type hetzelfde meoten zijn. Hieronder staat een voorbeeld waarbij t1 een integerbased tensor is en t2 een floatbased tensor. Die twee variabelen mag je dus niet optellen. Hetzelfde geldt voor devices, maar ik heb geen gpu dus dat kan ik niet testen.

In [6]:
t1 = torch.tensor([1, 2, 3])
t2 = torch.tensor([1., 2., 3.])
print(t1.dtype, t2.dtype)
t1 + t2

torch.int64 torch.float32


RuntimeError: expected type torch.FloatTensor but got torch.LongTensor

## Creating options using data

Als we data willen inladen en converteren naar een tensor, dan zijn daar meerdere mogelijkheden voor. Aanschouw:

In [7]:
data = np.array([1, 2, 3])
t1 = torch.Tensor(data)
t2 = torch.tensor(data)
t3 = torch.as_tensor(data)
t4 = torch.from_numpy(data)
print("{}\n{}\n{}\n{}".format(t1, t2, t3, t4 ))

tensor([1., 2., 3.])
tensor([1, 2, 3], dtype=torch.int32)
tensor([1, 2, 3], dtype=torch.int32)
tensor([1, 2, 3], dtype=torch.int32)


Je ziet dat `torch.Tensor(data)` een andere output geeft dan de andere functies. Dit is een *class constructor* terwijl de andere *factory functions* zijn.

# Creating options without data
Genereer tensors met voorgeprogrammeerde functies.

In [8]:
print(torch.eye(2), "\n")
print(torch.zeros(2, 2), "\n")
print(torch.ones(2, 2), "\n")
print(torch.rand(2, 2), "\n")

tensor([[1., 0.],
        [0., 1.]]) 

tensor([[0., 0.],
        [0., 0.]]) 

tensor([[1., 1.],
        [1., 1.]]) 

tensor([[0.6412, 0.1458],
        [0.5279, 0.9315]]) 



# Creating PyTorch tensors - Best options
Hierboven was aangegeven welke verschillende manieren er zijn om data in te laden. 

Een van die verschillen waren de data types. `torch.Tensor()` gebruikt de defautl datatype (`torch.float32`). Bij de factory functions wordt de datatype overgenomen tenzij anders gespecificeerd wordt (gebruik argument `dtype =`). 

Een belangrijke eigenschap van deze functies is te zien in het volgende voorbeeld.

In [9]:
data = np.array([1, 2, 3])
t1 = torch.Tensor(data)
t2 = torch.tensor(data)
t3 = torch.as_tensor(data)
t4 = torch.from_numpy(data)

In [10]:
data[0] = 0
data[1] = 0
data[2] = 0
print("t1: {}".format(t1))
print("t2: {}".format(t2))
print("t3: {}".format(t3))
print("t4: {}".format(t4))

t1: tensor([1., 2., 3.])
t2: tensor([1, 2, 3], dtype=torch.int32)
t3: tensor([0, 0, 0], dtype=torch.int32)
t4: tensor([0, 0, 0], dtype=torch.int32)


Hier is te zien dat t1 en t2 de waardes hebben behouden. Terwijl t3 en t4 de veranderingen hebben overgenomen. Dit komt doordat `torch.Tensor()` en `torch.tensor()` een kopie maken van het geheugen. `torch.as_tensor()` en `torch.from_numpy()` delen het geheugen met de variabel waarvan ze een tensor hebben gemaakt. Het voordeel hiervan is de snelheid waarmee deze functies uitgevoerd kunnen worden. 

Dus met al deze verschillende opties. Welke moeten we nu gebruiken? Ligt aan de situatie. 
- All day use: `torch.tensor()`
- Tuning voor performence `torch.as_tensor()`

Het is goed om eerst je code te schrijven zodat die werkt met `torch.tensor()` en daarna code te optimaliseren en bottlenecks er uit halen met `torch.as_tensor()`

# Tensor operation types
De hoogste groepering van tensor operations vallen te groeperen in 4 soorten:
1. Reshaping operations
2. Element-wise operations
3. Reduction operations
4. Access operations
<br>
Reshaping operations zorgen ervoor dat we iets van houvast hebben als het gaat om de abstracte tensors. 

## Reshaping

In [11]:
t = torch.tensor([
    [1, 1, 1, 1],
    [2, 2, 2, 2],
    [3, 3, 3 ,3]
], dtype = torch.float32)
t.shape, len(t.shape)

(torch.Size([3, 4]), 2)

In [12]:
torch.tensor(t.shape).prod(), t.numel()

(tensor(12), 12)

Zoals eerder vermeld, moet de product van de shape gelijk staan aan het aantal elementen in de matrix. Je kan zelfs buiten de rank gaan. 

In [13]:
# Rank 2
t1_12 = t.reshape(1, 12)
t12_1 = t.reshape(12, 1)
t3_4 = t.reshape(3, 4)
t4_3 = t.reshape(4, 3)
t2_6 = t.reshape(2, 6)
t6_2 = t.reshape(6, 2)

# Rank 3
t2_2_3 = t.reshape(2, 2, 3)
t3_2_2 = t.reshape(3, 2, 2)
t2_3_2 = t.reshape(2, 3, 2)

Squeezing verwijderd alle axis met lengte = 1. Unsqueezing doet het omgekeerde op bepaalde dimensie.

In [14]:
print(t1_12.shape)
print(t1_12.squeeze())
print(t1_12.squeeze().shape)
print(t1_12.squeeze().unsqueeze(dim = 0).shape)

torch.Size([1, 12])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])
torch.Size([12])
torch.Size([1, 12])


In [15]:
def flatten(t):
    t = t.reshape(1, -1)
    t = t.squeeze()
    return t

In [17]:
flatten(t)

tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])

In [20]:
t.reshape(12)

tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])

# CNN Flatten Operation Visualized - Tensor Batch Processing for deep learning

Laten we 3 hypothetische afbeeldingen genereren. Deze zetten we gelijk in een tensor waarbij elke afbeelding een batch element is (A0).

In [24]:
t1 = torch.ones(4, 4)
t2 = torch.ones(4, 4)*2
t3 = torch.ones(4, 4)*3

In [38]:
t = torch.stack((t1, t2, t3))
t.shape #Size([batch, hight, width]) Size([A0, A2, A3])

torch.Size([3, 4, 4])

Omdat PyTorch een kleuren kanaal vereist, moeten we onze tensor reshapen. Tussen A0 en A2 moet A1 geplaatst worden. Op het moment hebben we maar 1 kleuren kanaal (grayscale).

In [40]:
t = t.reshape(3, 1, 4, 4)
t

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]],


        [[[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]]],


        [[[3., 3., 3., 3.],
          [3., 3., 3., 3.],
          [3., 3., 3., 3.],
          [3., 3., 3., 3.]]]])

In [41]:
print(t[0]) # Get first batch element
print(t[0][0]) # Get first color channel
print(t[0][0][0]) # Get first row
print(t[0][0][0][0]) # Get first col

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([1., 1., 1., 1.])
tensor(1.)


Flattening t op verschillende manieren:

In [37]:
print(t.reshape(1, -1)[0])
print(t.reshape(-1))
print(t.view(t.numel()))
print(t.flatten())

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])


Het vervelende hiervan is dat onze batches verdwenen zijn. Onze 3 afbeeldingen wordt nu gezien als een grote afbeelding. Dit maakt het onmogelijk om te trainen. Hieronder de oplossing.

In [45]:
print(t.flatten(start_dim=1))
print(t.flatten(start_dim=1).shape)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]])
torch.Size([3, 16])


In [47]:
print(t.reshape(3, -1))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]])


# Element-/Component-/Point-wise operations
## Element-wise operations
Een element-wise (ook wel component- of point-wise) operatie is een operatie tussen twee tensoren waar gebruikt wordt gemaakt van elementen op dezelfde positie in die tensoren. <br>
$\begin{pmatrix}
0\\
1\\
2\\
3
\end{pmatrix} \times
\begin{pmatrix}
3\\
2\\
1\\
0
\end{pmatrix} = 4 \rightarrow$ element wise <br>
$\begin{pmatrix}
0\\
1\\
2\\
3
\end{pmatrix} +
\begin{pmatrix}
3\\
2\\
1\\
0
\end{pmatrix} = 
\begin{pmatrix}
4\\
3\\
3\\
4
\end{pmatrix} \rightarrow$ element wise <br>
$\begin{pmatrix}
0\\
1\\
2\\
3
\end{pmatrix}
\begin{pmatrix}
3 & 2 & 1 & 0
\end{pmatrix} = 4 \rightarrow$ non element wise

In [49]:
t1 = torch.tensor([
    [1, 2],
    [3, 4]
], dtype = torch.float32)
t2 = torch.tensor([
    [9, 8],
    [7, 6]
],  dtype = torch.float32)

Element wise operaties zijn dus alleen mogelijk op tensors met dezelfde Shape.

In [52]:
t1[0][0], t2[0][0] # dezelfde positie op een andere tensor

(tensor(1.), tensor(9.))

In [54]:
t1 + t2

tensor([[10., 10.],
        [10., 10.]])

 Het gekke is dat de volgende operations wel werken, maar niet dezelfde Shape hebben. Dit komt omdat deze tensors met andere shapes worden gebroadcast wanneer mogelijk. Broadcasten kan ook worden gedaan door loops te schrijven, maar in numpy worden deze loops uitgevoerd in C zodat dit efficienter en dus sneller gebeurd. 

In [59]:
print(t1+2)
t3 = torch.tensor([2,4], dtype = torch.float32)
print(t1+t3)

tensor([[3., 4.],
        [5., 6.]])
tensor([[3., 6.],
        [5., 8.]])


In [67]:
print(t1 + torch.tensor(np.broadcast_to(2, t1.shape), dtype = torch.float32))
t3 = torch.tensor(np.broadcast_to(t3.numpy(), t1.shape))
print(t1 + t3)

tensor([[3., 4.],
        [5., 6.]])
tensor([[3., 6.],
        [5., 8.]])


## Comparison operations
Een tensor met dezelfde Shape wordt gereturned met elk element een 0 of 1.

In [68]:
t = torch.tensor([
    [0, 5, 7],
    [6, 0, 7],
    [0, 8, 0]
], dtype = torch.float32)

In [72]:
print(t.eq(0)) # equal
print(t.ge(0)) # greater or equal
print(t.gt(0)) # greater
print(t.le(0)) # less or equal
print(t.lt(0)) # less

tensor([[1, 0, 0],
        [0, 1, 0],
        [1, 0, 1]], dtype=torch.uint8)
tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype=torch.uint8)
tensor([[0, 1, 1],
        [1, 0, 1],
        [0, 1, 0]], dtype=torch.uint8)
tensor([[1, 0, 0],
        [0, 1, 0],
        [1, 0, 1]], dtype=torch.uint8)
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.uint8)


In [75]:
print(t.abs())
print(t.sqrt())
print(t.neg())
print(t.neg().abs())

tensor([[0., 5., 7.],
        [6., 0., 7.],
        [0., 8., 0.]])
tensor([[0.0000, 2.2361, 2.6458],
        [2.4495, 0.0000, 2.6458],
        [0.0000, 2.8284, 0.0000]])
tensor([[-0., -5., -7.],
        [-6., -0., -7.],
        [-0., -8., -0.]])
tensor([[0., 5., 7.],
        [6., 0., 7.],
        [0., 8., 0.]])


Reductie operations zijn bijna gelijk als element-wise operaties. Het verschil is dat de functie binnen 1 tensor wordt uitgevoerd ipv met een andere tensor.

In [79]:
t = torch.tensor([
    [0, 1, 0],
    [2, 0, 2],
    [0, 3, 0]
], dtype = torch.float32)
print(t.sum())
print(t.sum().numel() < t.numel()) # Het aantal elementen is verminderd door de sum functie

tensor(8.)
True


In [81]:
print(t.prod())
print(t.mean())
print(t.std())

tensor(0.)
tensor(0.8889)
tensor(1.1667)


In [83]:
t = torch.tensor([
    [1, 1, 1, 1],
    [2, 2, 2, 2],
    [3, 3, 3 ,3]
], dtype = torch.float32)
print(t.sum(dim = 0))
print(t.sum(dim = 1))

tensor([6., 6., 6., 6.])
tensor([ 4.,  8., 12.])


In [85]:
t.sum(dim = 0) ==  t[0] + t[1] + t[2]

tensor([1, 1, 1, 1], dtype=torch.uint8)

In [88]:
t.sum(dim = 1) ==  torch.tensor([t[0].sum(), t[1].sum(), t[2].sum()])

tensor([1, 1, 1], dtype=torch.uint8)

De ArgMax is een reductie functie die de index van de maximale waarde ophaalt. Als voorbeeld:

In [94]:
t = torch.tensor([
    [1, 0, 0, 2],
    [0, 3, 3, 0],
    [4, 0, 0, 5]
], dtype = torch.float32)
print(t.argmax())
print(t.max())
print(t.flatten())
print(t.flatten()[t.argmax()])

tensor(11)
tensor(5.)
tensor([1., 0., 0., 2., 0., 3., 3., 0., 4., 0., 0., 5.])
tensor(5.)


Wil je de maximale waarde van een bepaalde as? Gebruik dan wederom de "dim" argument. Bij `t.max(dim = 0)` wordend de indices automatisch gegeven.

In [100]:
print(t.max(dim = 0), t.argmax(dim = 0))
print(t.max(dim = 1), t.argmax(dim = 1))

(tensor([4., 3., 3., 5.]), tensor([2, 1, 1, 2])) tensor([2, 1, 1, 2])
(tensor([2., 3., 5.]), tensor([3, 2, 3])) tensor([3, 2, 3])


Om de waardes uit de tensor te halen gebruik je `.item` voor scalars en `.tolist` voor vectoren.

In [102]:
t = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype = torch.float32)
print(t.mean())
print(t.mean().item())

tensor(5.)
5.0


In [103]:
print(t.mean(dim = 0).tolist())
print(t.mean(dim = 0).numpy())

[4.0, 5.0, 6.0]
[4. 5. 6.]
